In [ ]:
import glob
import json
import shutil
import os
import cv2
import random
import albumentations as A

In [ ]:
# AI hub dataset

In [ ]:
json_list = glob.glob('./data/Training_라벨링데이터/**/*.json',recursive=True)

In [ ]:
category = ['가구류','나무','도기류','의류','자전거','전자제품','형광등']
cat_json_list = [[] for i in range(len(category))]

In [ ]:
label_to_path = './yolov8/data/labels/'

In [ ]:
#json -> yolo text file

def json_to_txt(label_list,to_path) : 

    for file_path in label_list :

        json_name = file_path.split('/')[-1]
        txt_name = json_name[:-5]+'.txt'
        cat_name = file_path.split('/')[4]

        with open(file_path,encoding='UTF8') as f:
            data = json.load(f)

            cat_num = category.index(cat_name)
            imgw = int(data['RESOLUTION'][:4])
            imgh = int(data['RESOLUTION'][-4:])

            yolo_format_list = []

            for dt in data['Bounding'] :
                draw_type = dt['Drawing']

                if draw_type == 'BOX' :
                    
                    cat_json_list[cat_num].append(file_path)

                    xcenter = int((int(dt['x1'])+int(dt['x2']))/2)/imgw
                    ycenter = int((int(dt['y1'])+int(dt['y2']))/2)/imgh
                    boxw = int((int(dt['x2'])-int(dt['x1'])))/imgw
                    boxh = int((int(dt['y2'])-int(dt['y1'])))/imgh

                    yolo_format = [cat_num,xcenter,ycenter,boxw,boxh]
                    yolo_format_list.append(yolo_format)

            with open(to_path+txt_name,'w',newline='') as fw :
                write = csv.writer(fw,delimiter=' ')
                write.writerows(yolo_format_list)



In [ ]:
json_to_txt(json_list,label_to_path)

In [ ]:
#sampling

def sampling (li,num) :
    if len(li) > num :
        sample_num = num
    else:
        sample_num = len(li)
        
    sample_li = random.sample(li,sample_num)
    return sample_li

In [ ]:
#sampling 100 from each class
sample_num = [100 for i in range(len(cat_json_list))]
sample_json_list = list(map(sampling,cat_json_list,sample_num))

In [ ]:
img_to_path = './yolov8/data/images'

In [ ]:
# resize img
def resize_img(img_path,width=800) :
    img = cv2.imread(img_path)
    h,w = img.shape[:2]
    ratio = w/h
    new_h = int(width/ratio)
    img = cv2.resize(img,(width,new_h),interpolation=cv2.INTER_AREA)
    return img

In [ ]:
def set_img(target_list, to_path) :
    for target_file in target_list:
        file_name = target_file.split('/')[-1]
        img_name = file_name.replace('txt','jpg')
        img_path = './data/images/'
        
        resized = resize_img(img_path+img_name)
        cv2.imwrite(to_path+img_name,resized)

In [ ]:
set_img(sample_json_list,img_to_path)

In [ ]:
#augmentation (albumentation)

def aug_img (label_list, target_number,datatype) :
    transform = A.Compose([
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3,p=1),])
    
    k= 0
    
    
    for label_files in label_list :
        
        org_len = len(label_files)
        i = 0
        
        while i + org_len < target_number :
            print(k)
            target_file = random.choice(label_files).replace('\\','/')
            target_file_name = target_file.split('/')[-1].replace('.txt','')
            img_name = target_file_name+'.jpg'
            img_target = img_from+img_name
            print(target_file)
            print(img_target)
            
            img = cv2.imread(img_target)
            transformed_image = transform(image=img)['image']
            
            if not os.path.exists(img_to+datatype+'_aug/') :
                os.makedirs(img_to+datatype+'_aug/')
            
            print(img_to+datatype+'_aug/'+target_file_name+'_aug_'+str(i)+'.jpg')
            cv2.imwrite(img_to+datatype+'_aug/'+target_file_name+'_aug_'+str(i)+'.jpg', transformed_image)
            
           
        
            #copy label
        
            if not os.path.exists(label_to+datatype+'_aug/') :
                os.makedirs(label_to+datatype+'_aug/')
            
            print(target_file,label_to+datatype+'_aug/'+target_file_name+'_aug_'+str(i)+'.txt')
            shutil.copy(target_file,label_to+datatype+'_aug/'+target_file_name+'_aug_'+str(i)+'.txt')
            
            
            i += 1
            k += 1

In [ ]:
aug_img(sample_json_list,160,'train')

In [ ]:
#selectstar dataset

In [ ]:
ss_json_list = glob.glob('./data/reusable-trash-images/reusable-trash-images/selectstar-reusable-trash-json/*.json')

In [ ]:
ss_labels = []
ss_shape_type = []
ss_max_h = 0
ss_min_h = 10000
ss_max_w = 0
ss_min_w = 10000

In [ ]:
class_count = {}
class_label_list = {}

In [ ]:
for idx,js in enumerate(ss_json_list):
    print(idx,js)
    with open(js,encoding='UTF8') as f:
        data = json.load(f)

        
        #label 1 only
        
        if len(data['shapes']) > 1:
            continue
        
        
        
        for dt in data['shapes'] :


            img_label = dt['label']
            [[x1,y1],[x2,y2]] = dt['points']
            shape_type = dt['shape_type']

            if img_label not in class_count :
                class_count[img_label] = 1
                class_label_list[img_label] = [js]
            else:
                class_count[img_label] +=1
                class_label_list[img_label].append(js)

            if shape_type not in ss_shape_type :
                ss_shape_type.append(shape_type)

        img_path = data['imagePath']
        img_w = data['imageWidth']
        img_h = data['imageHeight']


        if img_w > ss_max_w :
            ss_max_w = img_w

        if img_w < ss_min_w :
            ss_min_w = img_w

        if img_h > ss_max_h :
            ss_max_h = img_h

        if img_h < ss_min_h:
            ss_min_h = img_h


In [ ]:
# image/label -> each class folder

In [ ]:
for key,value in class_label_list.items() :
    for file in value :
        json_name = file.split('/')[-1]
        jpg_name = json_name.replace('json','jpg')
        
        label_to = 'g:/prj/data/ss/ss_labels/'+key+'/'
        img_to = 'g:/prj/data/ss/ss_images/'+key+'/'
        
        if not os.path.exists(label_to) :
            os.makedirs(label_to)
            
        if not os.path.exists(img_to) :
            os.makedirs(img_to)
            
        shutil.copy(file,label_to+json_name)
        shutil.copy('g:/prj/data/reusable-trash-images/reusable-trash-images/selectstar-reusable-trash-image/'+jpg_name,img_to+jpg_name)
    

In [ ]:
# TACO datasets

In [ ]:
taco_path = './data/taco/'
taco_label_to = taco_path+'labels/'
taco_img_to = taco_path+'images/'

In [ ]:
scat_map = {0:'0-2',1:'1-9',2:'2-17',3:'3-19',4:'4-27',5:'5-36',6:'6-43',7:'7-46'}

In [ ]:
cat_map = {0:[52],1:[38,41],2:[57],3:[6,23,26],4:[14,15,16,17,18,19,20,30,32,33,34],5:[10,11,12],6:[5],7:[4,21,24,29,39,45,47,49,55]}

In [ ]:
meta_df = pd.read_csv(taco_path+'meta_df.csv')

In [ ]:
taco_img_list = glob.glob(taco_path+'images/**/*.jpg',recursive=True)

In [ ]:
for i in range(4784) :
    img_file = str(meta_df.iloc[i][3])
    supercategory = str(meta_df.iloc[i][6])
    cat_name = str(meta_df.iloc[i][5])
    ann_id = str(meta_df.iloc[i][7])
    cat_id = str(meta_df.iloc[i][4])
    x1 = int(meta_df.iloc[i][8])
    y1 = int(meta_df.iloc[i][9])
    width = int(meta_df.iloc[i][10])
    height = int(meta_df.iloc[i][11])
    img_width = int(meta_df.iloc[i][1])
    img_height = int(meta_df.iloc[i][2])
    
    xcenter = (x1+width/2)/img_width
    ycenter = (y1+height/2)/img_height
    boxw = width/img_width
    boxh = height/img_height
    
    new_path = img_to+supercategory+'/'+cat_id+'-'+cat_name+'/'
    
    file_name = ann_id+'.jpg'
    
    
       
    # create label
    
    label_to = taco_path+'labels/'
    
    for j in range(8):
 
        if int(cat_id) in cat_map[j] :
            class_num = j
            break
        class_num = 99
    
    if class_num == 99 :
        continue
        
    
    class_num = scat_map[class_num].split('-')[-1]
    
    print('cat_name:',cat_name,'cat_id:',cat_id,'class_num:',class_num)

    
    yolo_format_list = []
    
    yolo_format = [class_num,xcenter,ycenter,boxw,boxh]
    yolo_format_list.append(yolo_format)

    
    
    if not os.path.exists(label_to):
        os.makedirs(label_to)
        

    with open(label_to+ann_id+'.txt','w',newline='') as fw :
        write = csv.writer(fw,delimiter=' ')
        write.writerows(yolo_format_list)
    
    
    
     # copy image

    if not os.path.exists(new_path):
        os.makedirs(new_path)
        

    if img_width>800 :
        


        shutil.copy(data_from+img_file,new_path+file_name)
    else : 

        cv2.imwrite(new_path+file_name,img_resize(data_from+img_file))
    
 